In [14]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
import torch
import json

# BertTokenizer の読み込み
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')

# データセットの読み込み
dataset_path = './train_data/test.json'
with open(dataset_path, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

# データセットをトークン化し、入力テキストとラベルを含む辞書に変換
tokenized_dataset = []
for data in dataset:
    tokenized_data = tokenizer(
        data["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"  # PyTorch テンソルとして返す
    )
    tokenized_data["labels"] = torch.tensor(data["labels"], dtype=torch.float)  # ラベルを FloatTensor に変換
    print(tokenized_data)
    tokenized_dataset.append(tokenized_data)

# モデルの読み込み
model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese')

# トレーニングアーギュメントの設定
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    logging_dir='./logs',
)

# トレーナーの作成
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# トレーニングの実行
trainer.train()


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


{'input_ids': tensor([[    2,  6369,  5867,    23,  5815, 28498, 28501,   143,   260,    24,
             5,   910,  1819,    14,     6,   453,  2323,    11,   251,   198,
          4551,   755,   818,   453,   468,  7610,   874,  2567,    83,     7,
         16910,   648,   251,  1042,     8,    94,   173,     5,   269,   497,
           470,    26, 28453,   442,   367,    15,     6,   809,     1,   212,
           179, 12803,   251,   283,   842,   371,    14,    85,   311,    26,
         28461, 28447,     8,    19,   186,     5,  1146,   109,   496,  2870,
            14,   174,    19,   701,    12, 28498,   648,  1346,   475, 10556,
           496,  3349,    26, 28461, 16918, 28449,     8,     3,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: too many values to unpack (expected 2)